In [57]:
from ai_core.llm.base import create_chat_model

from langgraph.constants import START, END
from langgraph.graph import StateGraph, MessagesState

from langchain_core.tools import create_retriever_tool

from langgraph.prebuilt import create_react_agent


from IPython.display import Image, display


In [ ]:
model = create_chat_model(
    llm_api_key="sk-gapk-F14i5UwldXoRfXe6AGxpUSD9G-B0JOUR",
    llm_model="anthropic/claude-3-5-sonnet-20240620",
    # llm_model="openai/gpt-4o-2024-05-13"
)

In [ ]:
# Langgraph
graph_builder = StateGraph(MessagesState)

def chatbot(state: MessagesState):
    return {"messages": [model.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
graph.invoke({"messages": "삶의 의미란 무엇일까?"})["messages"][-1].pretty_print()

In [27]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key="ba3954fe-9cbb-4599-966b-20b04b5d3441",
    openai_api_base="https://aihub-api.sktelecom.com/aihub/v1/sandbox")

chroma = Chroma(
    collection_name="ds-dtseungbum-billing_granularify_system_requirements-text-4d94",
    embedding_function=embeddings,
    persist_directory="../ai_core/data/daisy_demo_chromadb")


In [29]:
retriever = chroma.as_retriever(saerch_type="similarity", k=4)

In [36]:
tool = create_retriever_tool(retriever=retriever, name="billing_granularify_system_requirements",
                             description="Description of billing granularify system requirements")

tools = [tool]

app = create_react_agent(model, tools)

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
def stream(msg):
    inputs = {"messages": [("user", msg)]}
    for s in app.stream(inputs):
        if "agent" in s:
            messages = s["agent"]["messages"]
        elif "tools" in s:
            messages = s["tools"]["messages"]
        else:
            messages = s["messages"]
    
        messages[-1].pretty_print()
    
stream("과금세분화를 진행하게된 배경을 알 수 있을까?")

In [ ]:
stream("유레카라는 시스템이 하던 역할은 뭐야?")